# Task 1.2 Data Analysis Project City Bike NYC

## Goals
This dataset contains a sample of bike trips from the Citi Bike system in New York City.
Each row represents one trip and includes information about the start and end stations, the duration, the
user type, and other contextual data like age, season, temperature, and weekday.
Your goal is to explore this dataset and extract insights through data analysis with pandas.

You'll practice basic pandas operations (loading, exploring, cleaning, transforming, summarizing) and use descriptive statistics and simple visualizations to support your answers.

In [2]:
import pandas as pd
import numpy as np

pd.read_excel("ny_citibikes_raw.xlsx").to_csv("ny_citibikes_raw.csv", index=False)
df = pd.read_csv("ny_citibikes_raw.csv")
df.head()


,Start Time,Stop Time,Start Station ID,Start Station Name,End Station ID,End Station Name,Bike ID,User Type,Birth Year,Age,Age Groups,Trip Duration,Trip_Duration_in_min,Month,Season,Temperature,Weekday
0,2017-01-01 00:38:00,2017-01-01 01:03:00,3194,McGinley Square,3271,Danforth Light Rail,24668,Subscriber,1961,60,55-64,1513,25,1,Winter,10,Sunday
1,2017-01-01 01:47:00,2017-01-01 01:58:00,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
2,2017-01-01 01:47:00,2017-01-01 01:58:00,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
3,2017-01-01 01:56:00,2017-01-01 02:00:00,3186,Grove St PATH,3270,Jersey & 6th St,24604,Subscriber,1970,51,45-54,258,4,1,Winter,10,Sunday
4,2017-01-01 02:12:00,2017-01-01 02:23:00,3270,Jersey & 6th St,3206,Hilltop,24641,Subscriber,1978,43,35-44,663,11,1,Winter,10,Sunday


## 1. Dataset Exploration
- What information does each column contain?

Con los comandos df.head() y df.info() nos podemos hacer una idea de los contenidos de cada columna. Viendo los resultados se puede hacer un resumen más "humanizado" de cada columna:

- **Start Time**: guarda la fecha y hora de inicio del viaje como Object.
- **Stop Time**: guarda la fecha y hora de finalizacion del viaje como  Object.
- **Start Station ID**: guarda el identificador de la estación de comienzo del viaje como Int64.
- **Start Station Name**: guarda el nombre de la estación de comienzo del viaje como Object.
- **End Station ID**: guarda el identificador de la estación de finalización del viaje como Int64.
- **End Station Name**: guarda el nombre de la estación de finalización del viaje como Object.
- **Bike ID**: guarda el identificador de la bicicleta usada en el viaje como Int64.
- **User Type**: guarda el tipo de usuario que realiza el viaje como Object.
- **Birth Year**: guarda el año de nacimiento del usuario que realiza el viaje como Int64.
- **Age**: guarda la edad del usuario que realiza el viaje como Int64.
- **Age Groups**: guarda el intervalo de edad al que pertenece el usuario que realiza el viaje como Object.
- **Trip Duration**: guarda la duración del viaje en segundos como Int64.
- **Trip_Duration_in_min**: guarda la duración del viaje en minutos como Int64.
- **Month**: guarda el número que corresponde al mes en el que se realiza el viaje como Int64.
- **Season**: guarda la estación del año en la que se realiza el viaje como Object.
- **Temperature**: guarda la temperatura en grados celsius del momento en el que se realiza el viaje como Int64.
- **Weekday**: guarda el día de la semana en el que se realiza el viaje como Object.

Con esta información podemos hacernos una idea de cómo sacar los datos para analizarlos y que datos hay que limpiar u obviar. Si queda alguna duda siempre podemos indagar un poco más en la columna que no se entienda del todo. Por ejemplo quiero saber que datos se guardan en user type, porque a primera vista parece que se podría manejar con una columna "Subscriber" con resultados 0 y 1. Con un value_counts() puedo visualizar los datos que guarda, y con ello puedo sacar la conclusión de que si que se podría manejar con un booleano o un int que maneje 0 o 1.


In [3]:
df.info()
df["User Type"].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20400 entries, 0 to 20399
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Start Time            20400 non-null  object
 1   Stop Time             20400 non-null  object
 2   Start Station ID      20400 non-null  int64 
 3   Start Station Name    20400 non-null  object
 4   End Station ID        20400 non-null  int64 
 5   End Station Name      20399 non-null  object
 6   Bike ID               20400 non-null  int64 
 7   User Type             20400 non-null  object
 8   Birth Year            20400 non-null  int64 
 9   Age                   20400 non-null  int64 
 10  Age Groups            20400 non-null  object
 11  Trip Duration         20400 non-null  int64 
 12  Trip_Duration_in_min  20400 non-null  int64 
 13  Month                 20400 non-null  int64 
 14  Season                20400 non-null  object
 15  Temperature           20400 non-null

User Type
Subscriber       20020
One-time user      380
Name: count, dtype: int64

- Are there missing or duplicated values?

Si, hay un total de 3555 filas duplicadas en el dataset, estos registros se deben eliminar para no alterar las estadísticas con un volumen de datos que no es realista.

 Después de comprobar los duplicados se pueden comprobar los nulos que hay por columna. En este caso nos da un nulo en un nombre de la estación de llegada. Como no nos da el mismo número en el ID de la estación, podemos buscar si el ID coincide con alguna otra estación que se haya registrado.

In [4]:
print(f"Número de duplicados antes de la eliminación: {df.duplicated().sum()}\n")
df = df.drop_duplicates()
print(f"Número de duplicados después de la eliminación: {df.duplicated().sum()}\n")

# Comprobamos los nulos por columna:
print(f"Número de nulos por columna:\n{df.isna().sum()}\n")

# Buscamos si el nulo en el nombre de la estación tiene un ID repetido en alguna otra entrada.
print(df[df['End Station Name'].isna()])
print(df[df['End Station ID'] == 3211])

# Una vez comprobado podemos rellenar ese nulo con el nombre de la estación asociada al id, en este caso "Newark Ave".
df.loc[9858, 'End Station Name'] = 'Newark Ave'

# Volvemos a comprobar los nulos para ver si el campo se ha cubierto correctamente.
print(f"Número de nulos por columna: {df.isna().sum()}\n")

# Mostramos el campo con el nombre de la estación cambiada.
print(df.loc[9858])

Número de duplicados antes de la eliminación: 3555

Número de duplicados después de la eliminación: 0

Número de nulos por columna:
Start Time              0
Stop Time               0
Start Station ID        0
Start Station Name      0
End Station ID          0
End Station Name        1
Bike ID                 0
User Type               0
Birth Year              0
Age                     0
Age Groups              0
Trip Duration           0
Trip_Duration_in_min    0
Month                   0
Season                  0
Temperature             0
Weekday                 0
dtype: int64

               Start Time            Stop Time  Start Station ID  \
9858  2017-02-25 12:36:00  2017-02-25 12:44:00              3220   

     Start Station Name  End Station ID End Station Name  Bike ID   User Type  \
9858  5 Corners Library            3211              NaN    24522  Subscriber   

      Birth Year  Age Age Groups  Trip Duration  Trip_Duration_in_min  Month  \
9858        1963   58      55-64

- What is the overall time span of the trips?

El tiempo total empleado en los viajes es de 161.346 minutos. Que a su vez son 2.689,1 horas, que son 112 días en total.

In [20]:
print(f"Tiempo total empleado en los viajes: {sum(df['Trip_Duration_in_min'])} minutos.")

Tiempo total empleado en los viajes: 161346 minutos.


## 2. Basic Statistics
- What is the average trip duration (in minutes)?

La duración media de los viajes en minutos es aproximado a 9.58 mins/viaje. (Exactamente: 9.578272484416742 minutos)

In [6]:
print(f"Duración media en minutos por viaje: {df['Trip_Duration_in_min'].mean()}")

Duración media en minutos por viaje: 9.578272484416742


- What is the minimum and maximum duration?

La duración del viaje más corto es 1 minuto y la duración del viaje más largo son 6515 minutos, que son 109 horas que a su vez son 4 días y medio.

In [7]:
print(f"Duración mínima: {df['Trip_Duration_in_min'].min()}")
print(f"Duración máxima: {df['Trip_Duration_in_min'].max()}")

Duración mínima: 1
Duración máxima: 6515


- What are the most common start and end stations?

Ambos resultados pertenecen a la estación de **Grove St PATH** con un total de 4858 registros. 2115 registros como estación de salida y 2743 registros como estación de llegada.

In [8]:
print(f"\n\nEstación de salida más frecuentada: \n{df['Start Station Name'].value_counts().head(1)}")
print(f"\n\nEstación de llegada más frecuentada: \n{df['End Station Name'].value_counts().head(1)}")



Estación de salida más frecuentada: 
Start Station Name
Grove St PATH    2115
Name: count, dtype: int64


Estación de llegada más frecuentada: 
End Station Name
Grove St PATH    2743
Name: count, dtype: int64


## 3. Users and Demographics
- How many unique bikes were used?

Se han usado un total de 500 bicicletas en un total de 16845 viajes.

In [9]:
print(f"Número de bicis únicas usadas: {df['Bike ID'].nunique()}\n\n")
df.info()

Número de bicis únicas usadas: 500


<class 'pandas.core.frame.DataFrame'>
Index: 16845 entries, 0 to 20399
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Start Time            16845 non-null  object
 1   Stop Time             16845 non-null  object
 2   Start Station ID      16845 non-null  int64 
 3   Start Station Name    16845 non-null  object
 4   End Station ID        16845 non-null  int64 
 5   End Station Name      16845 non-null  object
 6   Bike ID               16845 non-null  int64 
 7   User Type             16845 non-null  object
 8   Birth Year            16845 non-null  int64 
 9   Age                   16845 non-null  int64 
 10  Age Groups            16845 non-null  object
 11  Trip Duration         16845 non-null  int64 
 12  Trip_Duration_in_min  16845 non-null  int64 
 13  Month                 16845 non-null  int64 
 14  Season                16845 non-null  object
 15  Temp

- What are the proportions of user types (Subscriber vs Customer)?

Hay un 98% de subscriptores del servicio usando las bicis, y un 1.89% de usuarios que no están suscritos al servicio.

In [10]:
usuarios = df['User Type'].value_counts()
print(usuarios / usuarios.sum() *100)

User Type
Subscriber       98.112199
One-time user     1.887801
Name: count, dtype: float64


- What is the age distribution of the users? Which age group uses the service the most?

Los grupos de edad están agrupados cada nueve años, aunque hay varias excepciones. Se empieza a contar desde los 18 años hasta los 24 (6 años), a partir de los 24 se agrupan cada nueve años, es decir, del 25 al 34, del 35 al 44..., y así hasta el 74. Los usuarios con 75 años para arriba se recogen en el mismo grupo. Los usuarios que más utilizan los servicios están entre los 35 y 44 años de edad. Suman un total de 7698 usos, sólo de los usuarios en ese grupo de edad.

In [11]:
df['Age Groups'].value_counts()

Age Groups
35-44    7698
25-34    4002
45-54    2973
55-64    1448
65-74     615
75+        55
18-24      54
Name: count, dtype: int64

## 4. Temporal Analysis
- How does the number of trips vary by weekday?

El día con más usos es el Miércoles. Se puede observar que los días con más usos son los 5 primeros días de la semana. Esto puede estar relacionado con los horarios de trabajo de la ciudad. Esto se supone al ver que el fin de semana hay una bajada de casi 1000 registros por día.

In [12]:
df['Weekday'].value_counts()

Weekday
Wednesday    3301
Thursday     2953
Monday       2526
Tuesday      2460
Friday       2449
Saturday     1591
Sunday       1565
Name: count, dtype: int64

- Which month or season has the most rides?

El mes en el que más se ha usado el servicio es Marzo con 7174 usos. Aunque el mes se divide entre Invierno y Primavera, el dataset recoge todos los datos de Marzo como si fuesen en Primavera. Aún así, la primavera, que cuenta con el mes de más uso, no es capaz de ganar al invierno como estación con más usos con un total de 9671. En conclusión, según este dataset Invierno es la estación con más usos, pero hay que tener en cuenta que Invierno sólo recoge los datos de Enero y Febrero, y Primavera recoge todos los datos de Marzo, aunque sólo le pertenezcan 10 días del mismo (21 al 31 de Marzo). En resumen: no se puede obtener una conclusión sobre que estación tiene más registros porque faltan datos y hay errores en la clasificación.

In [13]:
print(df['Month'].value_counts().head())
print(df['Season'].value_counts().head())

Month
3    7174
2    5052
1    4619
Name: count, dtype: int64
Season
Winter    9671
Spring    7174
Name: count, dtype: int64


- What time of day do most trips start?

Las franjas a tener en cuenta son alrededor de las 8:30 AM, las 12:30 PM y las 17:45 PM. Con la conclusión anterior también se puede observar que las horas coinciden principalmente con la hora de entrada, de descando de comida y de salida de la mayoría de los trabajos. Por lo tanto se puede aproximar que esos tres horarios son los más habituados por los usuarios.

In [14]:
df['Start Time'].value_counts().head(10)

Start Time
2017-02-25 12:43:00    6
2017-03-29 08:23:00    6
2017-02-25 12:36:00    5
2017-03-21 17:46:00    5
2017-03-09 17:45:00    5
2017-03-09 08:45:00    5
2017-03-25 12:11:00    5
2017-03-06 08:18:00    5
2017-02-27 17:47:00    5
2017-03-08 08:23:00    5
Name: count, dtype: int64

## 5. Geographic Analysis
- Which station pairs (start → end) appear most often?

El trayecto de Hamilton Park a Grove St PATH es el más habituado entre los usuarios. Como curiosidad también se puede observar que el trayecto inverso, es decir, de Grove St PATH a Hamilton Park, también entra en el top 5 de los trayectos más repetidos. También se puede visualizar que el trayecto de Morris Canal a Exchange Place y viceversa también es muy transitado. Aunque este último cuente con 656 registros, no consigue superar al trayecto ganador que cuenta con 674 registros en total.

In [15]:
df[['Start Station Name', 'End Station Name']].value_counts().head()

Start Station Name  End Station Name
Hamilton Park       Grove St PATH       401
Morris Canal        Exchange Place      366
Dixon Mills         Grove St PATH       293
Exchange Place      Morris Canal        290
Grove St PATH       Hamilton Park       273
Name: count, dtype: int64

- Are there any stations that appear only as start or only as end stations?

Si. Las estaciones de 'W 45 St & 8 Ave', 'Warren St & Church St', 'E 15 St & 3 Ave', 'JCBS Depot', 'Indiana' y 'Broadway & W 36 St' son estaciones que no cuentan con registros como "Start Station Name" en este dataset.

In [16]:
startStations = df['Start Station Name'].unique()
endStations = df['End Station Name'].unique()

# El símbolo ~ denota negación al resultado de la función. 
estacionesUnicasSalida = startStations[~np.isin(startStations, endStations)]
estacionesUnicasLlegada = endStations[~np.isin(endStations, startStations)]

print("Solo en salidas:", estacionesUnicasSalida)
print("Solo en llegadas:", estacionesUnicasLlegada)

Solo en salidas: []
Solo en llegadas: ['W 45 St & 8 Ave' 'Warren St & Church St' 'E 15 St & 3 Ave' 'JCBS Depot'
 'Indiana' 'Broadway & W 36 St']


## 6. Temperature and Duration
- Is there any visible relationship between temperature and trip duration?

En esta tabla se puede comprobar la media de la duración de los viajes en minutos por cada temperatura recogida. Nos da una idea general de que están bastante repartidos los minutos. La temperatura que cuenta con más minutos de viaje es 11. Pero eso no quiere decir que esa temperatura sea la perfecta para viajar. Ese resultado se puede dar a causa de un dato atípico. Por lo tanto no se puede visualizar correctamente una relación entre la temperatura y la duración de los viajes.

In [17]:
df.groupby('Temperature', as_index=False)['Trip_Duration_in_min'].mean()

,Temperature,Trip_Duration_in_min
0,9,6.829787
1,10,8.668478
2,11,17.445338
3,12,7.571429
4,13,8.407113
5,14,10.355951
6,15,13.090503
7,16,8.972888
8,17,7.560858
9,18,7.162198


- How does average trip duration vary by season?

Tampoco se puede observar una diferencia muy obvia entre la duración de los viajes que se realizaron en invierno y lo que se realizaron en primavera. Por lo tanto también se podría descartar una relación.

In [18]:
df.groupby('Season', as_index=False)['Trip_Duration_in_min'].mean()

,Season,Trip_Duration_in_min
0,Spring,9.719264
1,Winter,9.473684


## 7. Summary and Interpretation
- Write a short summary (5–10 lines) of your findings.

En cada apartado he ido analizando brevemente los datos que se consultaban, y con eso puedo hacerme una idea general de los datos que incluye el dataset y, más importante, dónde y para qué aplicar estos datos. He podido llegar a la conclusión de que en este dataset no se pueden utilizar los datos de Season ni los datos de Month, al menos para hacer investigaciones que ocupen una parte seprior de tiempo, como por ejemplo un año o inlcuso medio año. Sin embargo si que se puede estudiar el rendimiento de cada estación y de las rutas entre ellas. También se pueden investigar las relaciones entre los grupos de edad y el número de registros para ayudar a incluir ciertos grupos que no participan tanto en el servicio. Y muchas más conclusiones y posibles mejoras que se podrían sacar con estos datos.

- Mention patterns, anomalies, or interesting trends you observed.

Personalmente me pareció interesante el número de viajes que se registraron entre las rutas de Hamilton Park a Grove St PATH y Morris Canal a Exchange Place. Esto nos podría ayudar a la hora de montar una nueva estación entre estas rutas para aumentar las posibilidades de parada de los usuarios que las usan. También podría servir para añadir más bicicletas en esta ruta o mejorar el estado de la carretera o carril para que los usuarios estén más protegidos ante accidentes.